# Ceneo Scraper

## Lista kroków "manualnych"

1. Wejdź na stronę produktu / z opiniami o produkcie
2. Dla każdej opinii na stronie
    1. Skpiuj opinię
    2. Wklej opinię do edytora tekstu
3. Przejź do klejnej strony z opiami
4. Powtarzaj kroki 2-3 dopóki są strony z opiniami  

## Lista kroków scrapera

1. Wysłanie żądania dostępu do zasobu - strona z opiniami o produkcie
2. Wydobycie z kodu HTML fragmentów odpowiadających opiniom
3. Dla każdej opinii na stronie
    1. Wydobycie z kodu opini jej składowych 

## Struktura opini w serwisie Ceneo.pl

|składowa|nazwa|selektor|
|--------|-----|--------|
|identyfikator opinii|opinion_id||
|autor|author||
|rekomendacja|recommendation||
|treść opinii|content||
|lista zalet|pros||
|lista wad|cons||
|data wystawienia opinii|post_date||
|data zakupu produktu|purchase_date||
|ile osób uznało opinię za przydatną|useful||
|ile osób uznało opinię za nieprzydatną|useless||

63717975
124893467
108481121
114228736
28020568

1. Import bibliotek

In [ ]:
import requests
from bs4 import BeautifulSoup

2. Wysłanie żądania dostępu do zasobu (request)

In [ ]:
product_code = "28020568"
url = f"https://www.ceneo.pl/{product_code}#tab=reviews"
response = requests.get(url)

3. Wydobycie z kodu HTML fragmentów odpowiadających opiniom

In [ ]:
page = BeautifulSoup(response.text, "html.parser")
opinions = page.select("div.js_product-review")


4. Wydobycie z kodu opini jej składowych dla każdej opinii na stronie

In [ ]:
for opinion in opinions:
    pass